### 豆瓣评分的预测

在这个项目中，我们要预测一部电影的评分，这个问题实际上就是一个分类问题。给定的输入为一段文本，输出为具体的评分。 在这个项目中，我们需要做：
- 文本的预处理，如停用词的过滤，低频词的过滤，特殊符号的过滤等
- 文本转化成向量，将使用三种方式，分别为tf-idf, word2vec以及BERT向量。 
- 训练逻辑回归和朴素贝叶斯模型，并做交叉验证
- 评估模型的准确率

在具体标记为``TODO``的部分填写相应的代码。 


In [12]:
#导入数据处理的基础包
import numpy as np
import pandas as pd

#导入用于计数的包
from collections import Counter

#导入tf-idf相关的包
from sklearn.feature_extraction.text import TfidfTransformer    
from sklearn.feature_extraction.text import CountVectorizer

#导入模型评估的包
from sklearn import metrics

#导入与word2vec相关的包
from gensim.models import KeyedVectors

#导入与bert embedding相关的包，关于mxnet包下载的注意事项参考实验手册
from bert_embedding import BertEmbedding
import mxnet

#包tqdm是用来对可迭代对象执行时生成一个进度条用以监视程序运行过程
from tqdm import tqdm

#导入其他一些功能包
import requests
import os

ImportError: this version of pandas is incompatible with numpy < 1.15.4
your numpy version is 1.14.6.
Please upgrade numpy to >= 1.15.4 to use this pandas version

### 1. 读取数据并做文本的处理
你需要完成以下几步操作：
- 去掉无用的字符如！&，可自行定义
- 中文分词
- 去掉低频词

In [1]:
import pandas as pd
#读取数据
data = pd.read_csv('data/DMSC.csv')
#观察数据格式
data.head()

,ID,Movie_Name_EN,Movie_Name_CN,Crawl_Date,Number,Username,Date,Star,Comment,Like
0,0,Avengers Age of Ultron,复仇者联盟2,2017-01-22,1,然潘,2015-05-13,3,连奥创都知道整容要去韩国。,2404
1,10,Avengers Age of Ultron,复仇者联盟2,2017-01-22,11,影志,2015-04-30,4,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,381
2,20,Avengers Age of Ultron,复仇者联盟2,2017-01-22,21,随时流感,2015-04-28,2,奥创弱爆了弱爆了弱爆了啊！！！！！！,120
3,30,Avengers Age of Ultron,复仇者联盟2,2017-01-22,31,乌鸦火堂,2015-05-08,4,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,30
4,40,Avengers Age of Ultron,复仇者联盟2,2017-01-22,41,办公室甜心,2015-05-10,5,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,16


In [2]:
#输出数据的一些相关信息
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212506 entries, 0 to 212505
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   ID             212506 non-null  int64 
 1   Movie_Name_EN  212506 non-null  object
 2   Movie_Name_CN  212506 non-null  object
 3   Crawl_Date     212506 non-null  object
 4   Number         212506 non-null  int64 
 5   Username       212496 non-null  object
 6   Date           212506 non-null  object
 7   Star           212506 non-null  int64 
 8   Comment        212506 non-null  object
 9   Like           212506 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 16.2+ MB


In [3]:
#只保留数据中我们需要的两列：Comment列和Star列
data = data[['Comment','Star']]
#观察新的数据的格式
data.head()

,Comment,Star
0,连奥创都知道整容要去韩国。,3
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,4
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,2
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,4
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,5


In [4]:
# 这里的star代表具体的评分。但在这个项目中，我们要预测的是正面还是负面。我们把评分为1和2的看作是负面，把评分为3，4，5的作为正面
data['Star']=(data.Star/3).astype(int)
data.head()

,Comment,Star
0,连奥创都知道整容要去韩国。,1
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,1
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,1
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,1


#### 任务1： 去掉一些无用的字符

In [6]:
import re
# TODO1: 去掉一些无用的字符，自行定一个字符集合，并从文本中去掉
def clean_text(input_str):
    s = re.sub(r'[^\w]', ' ', input_str)
    return s.strip()
data['Comment'] = data['Comment'].apply(clean_text)
data.head()

,Comment,Star
0,连奥创都知道整容要去韩国,1
1,一个没有黑暗面的人不值得信任 第二部剥去冗长的铺垫 开场即高潮 一直到结束 会有人觉得只...,1
2,奥创弱爆了弱爆了弱爆了啊,0
3,与第一集不同 承上启下 阴郁严肃 但也不会不好看啊 除非本来就不喜欢漫威电影 场面更加宏大 ...,1
4,看毕 我激动地对友人说 等等奥创要来毁灭台北怎么办厚 她拍了拍我肩膀 没事 反正你买了两份旅...,1


#### 任务2：使用结巴分词对文本做分词

In [7]:
# TODO2: 导入中文分词包jieba, 并用jieba对原始文本做分词
import jieba
from tqdm import tqdm
def comment_cut(content):
    # TODO: 使用结巴完成对每一个comment的分词
    seg_list  = [word for word in jieba.cut(content)]
    return " ".join(seg_list)

# 输出进度条
data['comment_processed'] = data['Comment'].apply(comment_cut)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.754 seconds.
Prefix dict has been built succesfully.


In [8]:
# 观察新的数据的格式
data.head()

,Comment,Star,comment_processed
0,连奥创都知道整容要去韩国,1,连 奥创 都 知道 整容 要 去 韩国
1,一个没有黑暗面的人不值得信任 第二部剥去冗长的铺垫 开场即高潮 一直到结束 会有人觉得只...,1,一个 没有 黑暗面 的 人 不 值得 信任 第二部 剥去 冗长 的 铺垫 开...
2,奥创弱爆了弱爆了弱爆了啊,0,奥创 弱 爆 了 弱 爆 了 弱 爆 了 啊
3,与第一集不同 承上启下 阴郁严肃 但也不会不好看啊 除非本来就不喜欢漫威电影 场面更加宏大 ...,1,与 第一集 不同 承上启下 阴郁 严肃 但 也 不会 不 好看 啊 除非 本...
4,看毕 我激动地对友人说 等等奥创要来毁灭台北怎么办厚 她拍了拍我肩膀 没事 反正你买了两份旅...,1,看毕 我 激动 地 对 友人 说 等等 奥创 要 来 毁灭 台北 怎么办 厚 她...


#### 任务3：设定停用词并去掉停用词

In [10]:
import os
# TODO3: 设定停用词并从文本中去掉停用词

# 下载中文停用词表至data/stopWord.json中，下载地址:https://github.com/goto456/stopwords/
if not os.path.exists('data/stopWord.json'):
    stopWord = requests.get("https://raw.githubusercontent.com/goto456/stopwords/master/cn_stopwords.txt")
    with open("data/stopWord.json", "wb") as f:
         f.write(stopWord.content)

# 读取下载的停用词表，并保存在列表中
with open("data/stopWord.json","r",encoding='utf-8') as f:
    stopWords = f.read().split("\n")  
    
    
# 去除停用词
def rm_stop_word(wordList):
    # your code, remove stop words
    wordList = [word for word in wordList.split() if word not in stopWords]
    return " ".join(wordList)

#这行代码中.progress_apply()函数的作用等同于.apply()函数的作用，只是写成.progress_apply()函数才能被tqdm包监控从而输出进度条。
data['comment_processed'] = data['comment_processed'].apply(rm_stop_word)

In [11]:
# 观察新的数据的格式
data.head()

,Comment,Star,comment_processed
0,连奥创都知道整容要去韩国,1,奥创 知道 整容 韩国
1,一个没有黑暗面的人不值得信任 第二部剥去冗长的铺垫 开场即高潮 一直到结束 会有人觉得只...,1,一个 没有 黑暗面 值得 信任 第二部 剥去 冗长 铺垫 开场 高潮 一直 结束 会 有人 ...
2,奥创弱爆了弱爆了弱爆了啊,0,奥创 弱 爆 弱 爆 弱 爆
3,与第一集不同 承上启下 阴郁严肃 但也不会不好看啊 除非本来就不喜欢漫威电影 场面更加宏大 ...,1,第一集 不同 承上启下 阴郁 严肃 不会 好看 本来 喜欢 漫威 电影 场面 更加 宏大 单...
4,看毕 我激动地对友人说 等等奥创要来毁灭台北怎么办厚 她拍了拍我肩膀 没事 反正你买了两份旅...,1,看毕 激动 友人 说 奥创 毁灭 台北 厚 拍了拍 肩膀 没事 反正 买 两份 旅行 保险 惹


#### 任务4：去掉低频词，出现次数少于10次的词去掉

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
# TODO4: 去除低频词, 去掉词频小于10的单词，并把结果存放在data['comment_processed']里

cv  = CountVectorizer(min_df=10)
cv.fit_transform(data['comment_processed'])

def rm_mindf_word(wordList):
    # your code, remove stop words
    wordList = [word for word in wordList.split() if word  in cv.vocabulary_.keys()]
    return " ".join(wordList)


data['comment_processed'] = data['comment_processed'].apply(rm_mindf_word)

In [14]:
cv.vocabulary_['好看']

5374

In [22]:
# 观察新的数据的格式
data.head()

,Comment,Star,comment_processed
0,连奥创都知道整容要去韩国,1,奥创 知道 整容 韩国
1,一个没有黑暗面的人不值得信任 第二部剥去冗长的铺垫 开场即高潮 一直到结束 会有人觉得只...,1,一个 没有 黑暗面 值得 信任 第二部 冗长 铺垫 开场 高潮 一直 结束 有人 觉得 动作...
2,奥创弱爆了弱爆了弱爆了啊,0,奥创
3,与第一集不同 承上启下 阴郁严肃 但也不会不好看啊 除非本来就不喜欢漫威电影 场面更加宏大 ...,1,第一集 不同 承上启下 阴郁 严肃 不会 好看 本来 喜欢 漫威 电影 场面 更加 宏大 团...
4,看毕 我激动地对友人说 等等奥创要来毁灭台北怎么办厚 她拍了拍我肩膀 没事 反正你买了两份旅...,1,激动 友人 奥创 毁灭 台北 肩膀 没事 反正 两份 旅行


In [23]:
final_data = data[data['comment_processed']!=""]
final_data

,Comment,Star,comment_processed
0,连奥创都知道整容要去韩国,1,奥创 知道 整容 韩国
1,一个没有黑暗面的人不值得信任 第二部剥去冗长的铺垫 开场即高潮 一直到结束 会有人觉得只...,1,一个 没有 黑暗面 值得 信任 第二部 冗长 铺垫 开场 高潮 一直 结束 有人 觉得 动作...
2,奥创弱爆了弱爆了弱爆了啊,0,奥创
3,与第一集不同 承上启下 阴郁严肃 但也不会不好看啊 除非本来就不喜欢漫威电影 场面更加宏大 ...,1,第一集 不同 承上启下 阴郁 严肃 不会 好看 本来 喜欢 漫威 电影 场面 更加 宏大 团...
4,看毕 我激动地对友人说 等等奥创要来毁灭台北怎么办厚 她拍了拍我肩膀 没事 反正你买了两份旅...,1,激动 友人 奥创 毁灭 台北 肩膀 没事 反正 两份 旅行
...,...,...,...
212501,里里外外我都打满分 太赞了,1,满分 太赞
212502,超棒 拟人超级像的 每个配角都好有戏 每个动物都好萌 想摸摸毛 fox一直叫bunny ...,1,超棒 拟人 超级 每个 配角 有戏 每个 动物 好萌 摸摸 fox 一直 bunny 胡萝卜...
212503,狐狸确实帅,1,狐狸 确实
212504,不负我望 超级好看,1,不负 超级 好看


In [26]:
final_data.to_csv('./clean_data_all.csv', encoding='utf-8', index=None)
final_data.loc[:,['comment_processed','Star']].to_csv('./clean_data.csv', encoding='utf-8', index=None)

### 2. 把文本分为训练集和测试集
选择语料库中的20%作为测试数据，剩下的作为训练数据

In [27]:
from sklearn.model_selection import train_test_split
# TODO5: 把数据分为训练集和测试集. comments_train（list)保存用于训练的文本，comments_test(list)保存用于测试的文本。 y_train, y_test是对应的标签（0、1）
X = final_data['comment_processed']
y = final_data['Star']
test_ratio = 0.2
comments_train, comments_test,y_train, y_test  = train_test_split(X, y, test_size=test_ratio)

In [63]:
type(comments_train)

pandas.core.series.Series

### 3. 把文本转换成向量的形式

在这个部分我们会采用三种不同的方式:
- 使用tf-idf向量
- 使用word2vec
- 使用bert向量

转换成向量之后，我们接着做模型的训练

#### 任务6：把文本转换成tf-idf向量

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
# TODO6: 把训练文本和测试文本转换成tf-idf向量。使用sklearn的feature_extraction.text.TfidfTransformer模块
#    请留意fit_transform和transform之间的区别。 常见的错误是在训练集和测试集上都使用 fit_transform，需要避免！ 
#    另外，可以留意一下结果是否为稀疏矩阵
tv = TfidfVectorizer(use_idf=True, smooth_idf=True, norm=None)
tfidf_train = tv.fit_transform(comments_train)
tfidf_test= tv.transform(comments_test)
print (tfidf_train.shape, tfidf_test.shape)  

(164441, 14764) (41111, 14764)


In [36]:
import pickle
pickle.dump(tfidf_train, open("tfidf_train_vectorizer.pickle", "wb"))
pickle.dump(tfidf_test, open("tfidf_test_vectorizer.pickle", "wb"))  

#### 任务7：把文本转换成word2vec向量

In [34]:
from gensim.models import KeyedVectors
# 由于训练出一个高效的word2vec词向量往往需要非常大的语料库与计算资源，所以我们通常不自己训练Wordvec词向量，而直接使用网上开源的已训练好的词向量。
# data/sgns.zhihu.word是从https://github.com/Embedding/Chinese-Word-Vectors下载到的预训练好的中文词向量文件
# 使用KeyedVectors.load_word2vec_format()函数加载预训练好的词向量文件
model = KeyedVectors.load_word2vec_format('data/sgns.zhihu.word')

In [43]:
#预训练词向量使用举例
model['今天']

array([-3.51068e-01,  2.57389e-01, -1.46752e-01, -4.45400e-03,
       -1.04235e-01,  3.72475e-01, -4.29349e-01, -2.80470e-02,
        1.56651e-01, -1.27600e-01, -1.68833e-01, -2.91350e-02,
        4.57850e-02, -3.53735e-01,  1.61205e-01, -1.82645e-01,
       -1.35340e-02, -2.42591e-01, -1.33356e-01, -1.31012e-01,
       -9.29500e-02, -1.70479e-01, -2.54004e-01, -1.20530e-01,
       -1.33690e-01,  7.84360e-02, -1.46603e-01, -2.77378e-01,
       -1.36723e-01,  9.29070e-02, -4.00197e-01,  2.80726e-01,
       -1.73282e-01,  8.56630e-02,  2.37251e-01,  6.24290e-02,
       -1.57132e-01,  2.15685e-01,  9.54770e-02,  1.09896e-01,
       -2.05394e-01, -3.37900e-03, -2.77480e-02,  8.16580e-02,
        9.65290e-02,  1.23188e-01,  9.55090e-02, -2.31017e-01,
       -8.59590e-02, -2.21634e-01, -1.37885e-01, -1.84790e-01,
       -2.40127e-01, -2.79150e-01, -4.56200e-03,  1.04099e-01,
        3.20523e-01, -6.77270e-02,  1.95719e-01,  4.06145e-01,
       -2.98546e-01, -1.67750e-02,  2.74917e-01, -9.023

In [41]:
model.vocab

{'，': <gensim.models.keyedvectors.Vocab at 0x27ee7eb8>,
 '的': <gensim.models.keyedvectors.Vocab at 0x28603f28>,
 '。': <gensim.models.keyedvectors.Vocab at 0x285f70f0>,
 '了': <gensim.models.keyedvectors.Vocab at 0x28d25e80>,
 '和': <gensim.models.keyedvectors.Vocab at 0x286a19e8>,
 '是': <gensim.models.keyedvectors.Vocab at 0x286a1b38>,
 '、': <gensim.models.keyedvectors.Vocab at 0x286a1a20>,
 '一个': <gensim.models.keyedvectors.Vocab at 0x286a1a58>,
 '我': <gensim.models.keyedvectors.Vocab at 0x286a1b70>,
 '有': <gensim.models.keyedvectors.Vocab at 0x286a1ba8>,
 '（': <gensim.models.keyedvectors.Vocab at 0x286a1be0>,
 '：': <gensim.models.keyedvectors.Vocab at 0x286a1c18>,
 '不': <gensim.models.keyedvectors.Vocab at 0x286a1c50>,
 '都': <gensim.models.keyedvectors.Vocab at 0x286a1c88>,
 '在': <gensim.models.keyedvectors.Vocab at 0x286a1cc0>,
 '上': <gensim.models.keyedvectors.Vocab at 0x286a1cf8>,
 '他': <gensim.models.keyedvectors.Vocab at 0x286a1d30>,
 '个': <gensim.models.keyedvectors.Vocab at 0x28

In [52]:
import numpy as np
# TODO7: 对于每个句子，生成句子的向量。具体的做法是：包含在句子中的所有单词的向量做平均。
vocabulary = model.vocab

def sentence_vec(sentence):
    ret_vec = np.zeros((300), dtype="float32")
    cnt = 0
    for word in sentence.split():
        if word in vocabulary:
            cnt += 1
            ret_vec = model[word]
    ret_vec /= cnt
    return ret_vec
    
word2vec_train = comments_train.apply(sentence_vec)
word2vec_test = comments_test.apply(sentence_vec)
print (word2vec_train.shape, word2vec_test.shape)

(164441,) (41111,)


In [53]:
import pickle
pickle.dump(word2vec_train, open("word2vec_train_vectorizer.pickle", "wb"))
pickle.dump(word2vec_test, open("word2vec_test_vectorizer.pickle", "wb"))  

#### 任务8：把文本转换成bert向量

In [12]:
data = []
label = []
with open('./clean_data_all.csv', encoding='utf-8') as f:
    for line in f:
        #print(line)
        items = line.strip().split(',')
        data.append(items[2])
        label.append(items[1])

In [17]:
from sklearn.model_selection import train_test_split
# TODO5: 把数据分为训练集和测试集. comments_train（list)保存用于训练的文本，comments_test(list)保存用于测试的文本。 y_train, y_test是对应的标签（0、1）

test_ratio = 0.2
comments_train, comments_test,y_train, y_test  = train_test_split(data, label, test_size=test_ratio)

In [3]:
from bert_embedding import BertEmbedding
# 导入gpu版本的bert embedding预训练的模型。
# 若没有gpu，则ctx可使用其默认值cpu(0)。但使用cpu会使程序运行的时间变得非常慢
# 若之前没有下载过bert embedding预训练的模型，执行此句时会花费一些时间来下载预训练的模型
#ctx = mxnet.gpu()
#embedding = BertEmbedding(ctx=ctx)
#bert_12_768_12  bert_24_1024_16
#embedding = BertEmbedding()
embedding = BertEmbedding(model='bert_12_768_12',dataset_name='wiki_cn')
# TODO8: 跟word2vec一样，计算出训练文本和测试文本的向量，仍然采用单词向量的平均。
# bert_train=
# bert_test=
# print (bert_train.shape, bert_test.shape)

C:\ProgramData\Anaconda3\lib\site-packages\gluonnlp\model\bert.py:693: UserWarning: wiki_cn/wiki_multilingual will be deprecated. Please use wiki_cn_cased/wiki_multilingual_uncased instead.
  warnings.warn('wiki_cn/wiki_multilingual will be deprecated.'


Vocab file is not found. Downloading.


In [ ]:
bert_train = embedding(comments_train,'avg')
bert_test = embedding(comments_test, 'avg')

In [64]:
print (tfidf_train.shape, tfidf_test.shape)
print (word2vec_train.shape, word2vec_test.shape)
#print (bert_train.shape, bert_test.shape)

(164441, 14764) (41111, 14764)
(164441,) (41111,)


### 4. 训练模型以及评估
对如上三种不同的向量表示法，分别训练逻辑回归模型，需要做：
- 搭建模型
- 训练模型（并做交叉验证）
- 输出最好的结果

In [ ]:
# 导入逻辑回归的包
from sklearn.linear_model import LogisticRegression

#### 任务9：使用tf-idf，并结合逻辑回归训练模型

In [ ]:
# TODO9: 使用tf-idf + 逻辑回归训练模型，需要用gridsearchCV做交叉验证，并选择最好的超参数



print('TF-IDF LR test accuracy %s' % metrics.accuracy_score(y_test, tf_idf_y_pred))
#逻辑回归模型在测试集上的F1_Score
print('TF-IDF LR test F1_score %s' % metrics.f1_score(y_test, tf_idf_y_pred,average="macro"))

#### 任务10：使用word2vec，并结合逻辑回归训练模型

In [ ]:
# TODO10: 使用word2vec + 逻辑回归训练模型，需要用gridsearchCV做交叉验证，并选择最好的超参数



print('Word2vec LR test accuracy %s' % metrics.accuracy_score(y_test, word2vec_y_pred))
#逻辑回归模型在测试集上的F1_Score
print('Word2vec LR test F1_score %s' % metrics.f1_score(y_test, word2vec_y_pred,average="macro"))

#### 任务11：使用bert，并结合逻辑回归训练模型

In [ ]:
# TODO11: 使用bert + 逻辑回归训练模型，需要用gridsearchCV做交叉验证，并选择最好的超参数



print('Bert LR test accuracy %s' % metrics.accuracy_score(y_test, bert_y_pred))
#逻辑回归模型在测试集上的F1_Score
print('Bert LR test F1_score %s' % metrics.f1_score(y_test, bert_y_pred,average="macro"))

#### 任务12：对于以上结果请做一下简单的总结，按照1，2，3，4提取几个关键点，包括：
- 结果说明什么问题？
- 接下来如何提高？

1.
2.
3.
4.
5.
6.